In [218]:
import pandas
import re

In [219]:
with open('everything2.txt', 'r', encoding="cp437") as f:
    text = f.read()

with open('Humanist.vol27.txt', 'r', encoding="cp437") as f:
    text27 = f.read()

with open('Humanist.vol32.txt', 'r', encoding="cp437") as f:
    text32 = f.read()


In [220]:
#Concat files
text = text + text32
#Creating list of Lines of "text"
lines = text.split("\n")

In [221]:
print(len(lines))

2723325


In [222]:
#Creating dict of diffent mail header types
patterns = {
    "1": [
        r"^\s*From: (?P<sender>.*)",
        r"^\s*Subject: (?P<subject>.*)",
        r"^\s*Date: (?P<date>.*)",
        r"^\s*X-Humanist: (?P<humanist>.*)"
    ],
    "2": [
        r"^\s*From - (?P<date>.*)",
        r"^\s*X-Mozilla-Status: (?P<data>.*)",
        r"^\s*X-Mozilla-Status2: (?P<data>.*)",
        r"^\s*Return-path: (?P<data>.*)"
    ],
    "3": [
        r"^\s*From (?P<head>.*)",
        r"^\s*Return-Path: (?P<sender>.*)",
        r"^\s*X-Original-To: (?P<subject>.*)"
    ],
    "4": [
        r"^\s*Return-Path: (?P<sender>.*)",
        r"^\s*X-Original-To: (?P<subject>.*)",
        r"^\s*Delivered-To: (?P<subject>.*)",
        r"^\s*Received: (?P<subject>.*)"
    ],
    "5": [
        r"^\s*Return-Path: (?P<sender>.*)",
        r"^\s*X-Spam-Checker-Version: (?P<subject>.*)"
    ]
}


In [223]:
#Function to get the key of the current pattern
def get_keys_from_value(patterns, val):
    return [k for k, v in patterns.items() if v == val]

In [224]:

#Version 5
#Function to get the header of the current mail as str
def get_header_content_sum(start):
    header_len = 0
    header_content_sum = ""
    header_content_current = ""
    for i in range(start,len(lines)):
        #Mail starts after Null-Line
        #if ("" in lines[i]):
        #    header_len = i
        #    return header_content_sum, header_len
        
        #Mail starts with part of line "Humanist Discussion Group, Vol. "
        if ("Humanist Discussion Group, Vol. " in lines[i]):
            header_len = i
            return header_content_sum, header_len
        #Mail starts aftre "Content-Transfer-Encoding: base64"
        if (("X-PPP-Vhost: " in lines[i]) & (lines[i+1] == "") & (lines[i+4] == "") & ("Humanist Discussion Group, Vol. " not in lines[i+5]) & ("Humanist Discussion Group, Vol. " not in lines[i+6]) & ("Humanist Discussion Group, Vol. " not in lines[i+7])):
            header_len = i
            header_len = header_len + 5
            return header_content_sum, header_len
        #Mail starts after "X-PPP-Vhost: digitalhumanities.org"
        if (("X-PPP-Vhost: " in lines[i]) & (lines[i+1] == "") & ("Humanist Discussion Group, Vol. " not in lines[i+2]) & ("Humanist Discussion Group, Vol. " not in lines[i+3]) & ("Humanist Discussion Group, Vol. " not in lines[i+4]) & ("Humanist Discussion Group, Vol. " not in lines[i+5]) & ("Humanist Discussion Group, Vol. " not in lines[i+6])):
            header_len = i
            header_len = header_len + 2
            return header_content_sum, header_len
        #Mail starts after "Errors-To: humanist-bounces@lists.digitalhumanities.org"
        if (("Errors-To: " in lines[i]) & (lines[i+1] == "") & ("Humanist Discussion Group, Vol. " not in lines[i+2]) & ("Humanist Discussion Group, Vol. " not in lines[i+3]) & ("Humanist Discussion Group, Vol. " not in lines[i+4]) & ("Humanist Discussion Group, Vol. " not in lines[i+5]) & ("Humanist Discussion Group, Vol. " not in lines[i+6])):
            header_len = i
            header_len = header_len + 2
            return header_content_sum, header_len
        header_len = header_len +1
        header_content_current = lines[i]
        header_content_sum = header_content_sum + "[/n]" + header_content_current
        header_content_current = ""
        


In [225]:
#Version 5
#Parse through lines
lines_with_patterns = []
count = 0
x_humanist = ""
mail_content_sum = ""
mail_content_current = ""
for i in range(len(lines)):
        for pattern in patterns.values():
            window_length = len(pattern)
            window = lines[i: i + window_length]
            lines_patterns = zip(window, pattern)
            matches = []
            for l, p in lines_patterns:
                m = re.match(p, l)
                if m:
                    matches.append(m.groupdict())
            if len(matches) == len(pattern):
                if ("From " in lines[i-1]): #Exclusion of recognizing pattern 3 and 4 for the same mail
                     continue
                keys = get_keys_from_value(patterns, pattern) #Get key of the current mail pattern
                header = list(map(int, keys))
                count = count +1

                if header[0] == 1:
                    lines_with_patterns.append( #Append a new row with current content to list
                        {
                            "mailFrom": lines[i],
                            "mailSubject": lines[i+1],
                            "mailDate": lines[i+2],
                            "mail_x-humanist": lines[i+3],
                            "mailPattern": keys,
                            "mailHeader_count": count,
                            "mailContent": mail_content_sum,
                            "mailMultiple": ""
                        }
                    )
                    if mail_content_sum != "":
                         lines_with_patterns[count-2]["mailContent"] = mail_content_sum #Adding content to a specific position in the list
                         mail_content_sum = ""

                if (header[0] == 2) | (header[0] == 3) | (header[0] == 4) | (header[0] == 5):
                    content_header_sum, header_len = get_header_content_sum(i)
                    if (header_len) < len(lines):
                         x_humanist = lines[header_len]
                    lines_with_patterns.append(
                        {
                            "mail_x-humanist": x_humanist,
                            "mailPattern": keys,
                            "mailHeader_count": count,
                            "mailHeader_content": content_header_sum,
                            "mailContent": mail_content_sum,
                            "mailMultiple": ""
                        }
                    )
                    x_humanist = ""
                    if mail_content_sum != "":
                         lines_with_patterns[count-2]["mailContent"] = mail_content_sum
                         mail_content_sum = ""

        if header[0] == 1: 
            mail_content_current = lines[i+5]
            mail_content_sum = mail_content_sum + "<br>" + mail_content_current #Sum up the lines to capture the content of the current mail
            mail_content_current = ""

        if (header[0] == 2) | (header[0] == 3) | (header[0] == 4) | (header[0] == 5):
            if header_len <= i:
                mail_content_current = lines[i]
                mail_content_sum = mail_content_sum + "<br>" + mail_content_current
                mail_content_current = ""
            
            if (i+1) == len(lines):
                         lines_with_patterns[count-1]["mailContent"] = mail_content_sum #Adding content to the last position in the list
                         mail_content_sum = ""
            
print(len(lines_with_patterns))

36462


In [226]:
#Creating Dataframe
dF = pandas.DataFrame(lines_with_patterns)

In [227]:
#List entries containing NaN are replaced by an empty string
dF.fillna('', inplace=True)

In [228]:
#Specific parts of list entries are replaced by an empty string
#dF["mailDate"] = dF["mailContent"].str.extract(REGEX, regex = True) 
dF["mailDate"] = dF["mailContent"].str.extract(r"Date: (.+?)<br>", expand=False)
dF["mailYear"] = dF["mailDate"].str.extract(r"\b(198[7-9]|199\d|200\d|201[0-8]|8[7-9]|9[0-9])\b", expand=False)


dF["mailFrom"] = dF["mailFrom"].replace(to_replace = "From: ", value ="", regex = True) #
dF["mailSubject"] = dF["mailSubject"].replace(to_replace = "Subject: ", value ="", regex = True)
dF["mailDate"] = dF["mailDate"].replace(to_replace = "Date: ", value ="", regex = True)
dF["mailContent"] = dF["mailContent"].replace(to_replace = "<javascript:", value ="", regex = True)
dF["mail_x-humanist"] = dF["mail_x-humanist"].replace(to_replace = "Humanist Discussion Group, ", value ="", regex = True)


In [229]:
dF.fillna('', inplace=True)
print(dF.loc[36051]["mailYear"])
referenceYear = "1987"
for i in range(len(dF)):
    year = dF.loc[i]["mailYear"]
    if (year == ""):
        dF.loc[i, "mailYear"] = referenceYear
    if (len(year) == 2):
        year = '19' + year
        referenceYear = year
        dF.loc[i, "mailYear"] = year
    if (len(year) == 4):
        referenceYear = year
    

2018


In [230]:
dF.loc[dF["mailDate"].str.contains(r"^<br>.*", regex=True), ["mailDate"]] = ""
dF.loc[dF['mailFrom'].isna() | (dF['mailFrom'] == ''), 'mailFrom'] = dF['mailContent'].str.extract(r"<([^<>]*@[^<>]*)>", expand=False)
#dF.loc[dF['mailFrom'] == 'MCCARTY@UTOREPAS', 'mailFrom'] = dF['mailContent'].str.extract(r"<([^<>]*@[^<>]*)>", expand=False)


In [231]:
dF["mailTimeZone"] = dF["mailDate"].str.extract(r"([+-]0\d{3}|[+-]1[0-8]\d{2})", expand=False)


In [232]:
dF.loc[dF['mailTimeZone'].isna() | (dF['mailTimeZone'] == ''), 'mailTimeZone'] = dF['mailDate'].str.extract(r"\b([A-Z]{3,4})\b", expand=False)

In [233]:
dF.loc[dF['mailTimeZone'].isna() | (dF['mailTimeZone'] == ''), 'mailTimeZone'] = dF['mailDate'].str.extract(r"\b([a-z]{3,4})\b", expand=False)

In [234]:
dF.fillna('', inplace=True)
dF.loc[dF["mailTimeZone"].str.contains(r"JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEZ", regex=True), ["mailTimeZone"]] = ""

In [235]:

dF.loc[dF['mailTimeZone'].isna() | (dF['mailTimeZone'] == ''), 'mailTimeZone'] = dF['mailDate'].str.extract(r"(?<=\d{2}:\d{2}\s)([A-Za-z]{2,3})", expand=False)

In [236]:
timezone_mapping = {
    'EST': '-0500',  # Eastern Standard Time
    'est': '-0500',  # Eastern Standard Time
    '-0500': '-0500',  # Eastern Standard Time
    'EDT': '-0400',  # Eastern Daylight Time
    'edt': '-0400',  # Eastern Daylight Time
    '-0400': '-0400',  # Eastern Daylight Time
    '+0400': '+0400', # Asia/Tehran
    'CST': '-0600',  # Central Standard Time
    'cst': '-0600',  # Central Standard Time
    '-0600': '-0600',  # Central Standard Time
    'CDT': '-0500',  # Central Daylight Time
    'cdt': '-0500',  # Central Daylight Time
    'MST': '-0700',  # Mountain Standard Time
    'mst': '-0700',  # Mountain Standard Time
    '-0700': '-0700',  # Mountain Standard Time
    '+0700': '+0700', # Asia/Bangkok
    'MDT': '-0600',  # Mountain Daylight Time
    'mdt': '-0600',  # Mountain Daylight Time
    'PST': '-0800',  # Pacific Standard Time
    'pst': '-0800',  # Pacific Standard Time
    '-0800': '-0800',  # Pacific Standard Time
    'PDT': '-0700',  # Pacific Daylight Time
    'pdt': '-0700',  # Pacific Daylight Time
    'GMT': '+0000',  # Greenwich Mean Time
    'gmt': '+0000',  # Greenwich Mean Time
    '+0000': '+0000',  # Greenwich Mean Time
    '-0000': '-0000',
    'UTC': '+0000',   # Coordinated Universal Time
    'utc': '+0000',   # Coordinated Universal Time
    'AST': '-0400',  # Atlantic Standard Time
    'ast': '-0400',  # Atlantic Standard Time
    'ADT': '-0300',  # Atlantic Daylight Time
    'adt': '-0300',  # Atlantic Daylight Time
    '-0300': '-0300',  # Atlantic Daylight Time
    'NST': '-0330',  # Newfoundland Standard Time
    'nst': '-0330',  # Newfoundland Standard Time
    '-0330': '-0330',  # Newfoundland Standard Time
    'NDT': '-0230',  # Newfoundland Daylight Time
    'ndt': '-0230',  # Newfoundland Daylight Time
    '-0230': '-0230',  # Newfoundland Daylight Time
    'AKST': '-0900',  # Alaska Standard Time
    'akst': '-0900',  # Alaska Standard Time
    '-0900': '-0900',  # Alaska Standard Time
    'AKDT': '-0800',  # Alaska Daylight Time
    'akdt': '-0800',  # Alaska Daylight Time
    'HST': '-1000',  # Hawaii Standard Time
    'hst': '-1000',  # Hawaii Standard Time
    '-1000': '-1000',  # Hawaii Standard Time
    'HDT': '-0900',  # Hawaii Daylight Time
    'hdt': '-0900',  # Hawaii Daylight Time
    'EMT': '+0100',
    'emt': '+0100',
    'BST': '+0100',  # British Summer Time
    'bst': '+0100',
    '+0100': '+0100',
    'BDT': '+0600',
    'bdt': '+0600',
    '+0600': '+0600',
    'IST': '+0530',  # Indian Standard Time
    'ist': '+0530',
    '+0530': '+0530',
    'CET': '+0100',  # Central European Time
    'cet': '+0100',
    'ITA': '+0200',
    'ita': '+0200',
    'MEZ': '+0200',
    'mez': '+0200', 
    'CEST': '+0200',  # Central European Summer Time
    'cest': '+0200',
    'EET': '+0200',  # Eastern European Time
    'eet': '+0200',
    '+0200': '+0200',
    '-0200': '-0200', #America/SaoPaulo
    'EEST': '+0300',  # Eastern European Summer Time
    'eest': '+0300',
    '+0300': '+0300',
    'JST': '+0900',  # Japan Standard Time
    'jst': '+0900',
    'KST': '+0900',  # Korea Standard Time
    'kst': '+0900',
    '+0900': '+0900',
    'AEST': '+1000',  # Australian Eastern Standard Time
    'aest': '+1000',
    '+1000': '+1000',
    'AEDT': '+1100',  # Australian Eastern Daylight Time
    'aedt': '+1100',
    '+1100': '+1100',
    'NZST': '+1200',  # New Zealand Standard Time
    'nzst': '+1200',
    '+1200': '+1200',
    'NZDT': '+1300',  # New Zealand Daylight Time
    'nzdt': '+1300',
    '+1300': '+1300',
    'MSK': '+0300',  # Moscow Time
    'msk': '+0300',
    'SGT': '+0800',  # Singapore Time
    'sgt': '+0800',
    'AWST': '+0800',  # Australian Western Standard Time
    'awst': '+0800',
    '+0800': '+0800',
    'ACST': '+0930',  # Australian Central Standard Time
    'acst': '+0930',
    '+0930': '+0930',
    'ACDT': '+1030',  # Australian Central Daylight Time
    'acdt': '+1030',
    '+1030': '+1030',
    'WAT': '+0100',  # West Africa Time
    'wat': '+0100',
    'CAT': '+0200',  # Central Africa Time
    'cat': '+0200',
    'SAST': '+0200',  # South African Standard Time
    'sast': '+0200',
    'FJT': '+1200',  # Fiji Time
    'fjt': '+1200',
    'LKT': '+0530',  # Sri Lanka Time
    'lkt': '+0530',
    'MVT': '+0500',  # Maldives Time
    'mvt': '+0500',
    '+0500': '+0500',
    'PHT': '+0800',  # Philippine Time
    'pht': '+0800',
    'IRKT': '+0800',  # Irkutsk Time
    'irkt': '+0800',
    'ULAT': '+0800',  # Ulaanbaatar Time
    'ulat': '+0800',
    'VET': '-0430',  # Venezuelan Standard Time
    'vet': '-0430',
    '-0430': '-0430',
    'CLT': '-0300',  # Chile Standard Time
    'clt': '-0300',
    'CLST': '-0300',  # Chile Summer Time
    'clst': '-0300',
    'ART': '-0300',  # Argentina Time
    'art': '-0300'
}

dF['mailTimeZone'] = dF['mailTimeZone'].map(timezone_mapping)

In [237]:
dF["mailVolume"] = dF["mail_x-humanist"].str.extract(r"(Vol\. \d{1,2})", expand=False)

In [238]:
#Search for multiple mails in mail_content and mark them with Y
dF.loc[dF["mailContent"].str.contains(r"\(1\) ---+", regex=True), ["mailMultiple"]] = "Y"
dF.loc[dF["mailContent"].str.contains(r"\(1\)---+", regex=True), ["mailMultiple"]] = "Y"
dF.loc[dF["mailContent"].str.contains(r"\[1\]---+", regex=True), ["mailMultiple"]] = "Y"

In [ ]:
dF = dF.applymap(str) 

In [239]:
dF.isnull().sum()

mailFrom                 0
mailSubject              0
mailDate                 0
mail_x-humanist          0
mailPattern              0
mailHeader_count         0
mailContent              0
mailMultiple             0
mailHeader_content       0
mailYear                 0
mailTimeZone          2583
mailVolume              16
dtype: int64

In [240]:
dF.dtypes

mailFrom              object
mailSubject           object
mailDate              object
mail_x-humanist       object
mailPattern           object
mailHeader_count       int64
mailContent           object
mailMultiple          object
mailHeader_content    object
mailYear              object
mailTimeZone          object
mailVolume            object
dtype: object

In [ ]:
y = []

for i in range(len(dF)):
    x = tuple(dF.iloc[i])
    y.append(x)

In [ ]:
file = open('emailarchives.txt', 'w')

for tuple in y:
    file.write(str(tuple) + ',' + '\n')

file.close()

In [241]:
print(dF.loc[16660]["mailYear"])

1997


In [242]:
cols = ['mailPattern', 'mailHeader_count']
dF.drop(cols, inplace=True, axis = 1)

In [243]:
dF.to_csv('batest9.csv')

In [ ]:
dF2 = pandas.DataFrame()

In [ ]:
dF2 = dF.head(500)

In [ ]:
dF2.to_csv('batest7.csv')

In [ ]:
dF.to_csv('batest6.csv')

In [ ]:
dF.head()

In [ ]:
dF2 = pandas.DataFrame()
dF3 = pandas.DataFrame()

In [ ]:
dF2 = dF.head(100)

In [ ]:
dF2.to_csv('batest4.csv')

In [ ]:
dF3  =dF.head(1000)

In [ ]:
dF3.to_csv('batest5.csv')

In [ ]:
cols = ['pattern', 'header_count', 'multiple_mails']
dF.drop(cols, inplace=True, axis = 1)

In [ ]:
dF.head()

In [ ]:
dF.dtypes

In [ ]:
dF.to_csv('batest2.csv')

In [ ]:
print(dF[dF["mailMultiple"]== "Y"].count())

In [ ]:
print(len(dF))
print(dF.loc[1001]["mailFrom"])
print(dF.loc[36051]["mailContent"])
print(dF.loc[36051]["mailYear"])

In [ ]:
print(lines_with_patterns[0]["from"])
print(lines_with_patterns[0]["date"])
print(lines_with_patterns[0])

lines_with_patterns[0]["date"] = "Test"
#lines_with_patterns[0].insert("date",("TEST"))
print(lines_with_patterns[0])
print(lines_with_patterns[0]["date"])
